In [ ]:
%pip install panel
%pip install ipyvuetify

In [ ]:
import requests
import json
import re
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [116]:
curl = input("请输入curl命令：")

您输入的内容是：


In [ ]:
token_match = re.search(r'token:\s*([a-zA-Z0-9]+)', curl)
ic_cookie_match = re.search(r'ic-cookie=([a-zA-Z0-9-]+)', curl)
token = token_match.group(1)
ic_cookie = ic_cookie_match.group(1)

In [ ]:
url = 'http://libzwyy.jlu.edu.cn/ic-web/reserve?roomIds=100652807&resvDates=20230627&sysKind=8'

headers = {
    'Host': 'libzwyy.jlu.edu.cn',
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'lan': '1',
    'token': token,
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.58',
    'Referer': 'http://libzwyy.jlu.edu.cn/',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6'
}

cookies = {
    'ic-cookie': ic_cookie,
}

response = requests.get(url, headers=headers, cookies=cookies)

if response.status_code == 200:
    data = json.loads(response.text)
    devices_data = data["data"]
else:
    print(f"Error: Request failed with status code {response.status_code}")

In [ ]:
# 提取 devName、startTime 和 endTime
table_data = []

for device in devices_data:
    dev_name = device["devName"]
    resv_info = device["resvInfo"]
    for resv in resv_info:
        start_time = datetime.fromtimestamp(resv["startTime"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        end_time = datetime.fromtimestamp(resv["endTime"] / 1000).strftime('%Y-%m-%d %H:%M:%S')
        table_data.append([dev_name, start_time, end_time])

In [ ]:
# 使用 pandas 创建 DataFrame
columns = ["devName", "startTime", "endTime"]
df = pd.DataFrame(table_data, columns=columns)

In [ ]:
df["startTime"] = pd.to_datetime(df["startTime"])
df["endTime"] = pd.to_datetime(df["endTime"])
df['duration'] = (df['endTime'] - df['startTime']).dt.seconds / 3600

In [ ]:
# 根据座位名称分组
groups = df.groupby("devName")

dev_names = df['devName'].unique()
name_to_index = {name: i for i, name in enumerate(dev_names)}

# 创建散点图
fig = go.Figure()

for name, group in groups:
    y_pos = name_to_index[name]
    for start_time, end_time in zip(group['startTime'], group['endTime']):
        fig.add_trace(
            go.Scatter(
                x=[start_time, end_time],
                y=[y_pos, y_pos],
                mode='lines',
                line=dict(color='orange', width=10),
                showlegend=False
            )
        )

# 设置x轴和y轴
fig.update_xaxes(title_text='时间', tickformat='%H:%M')
fig.update_yaxes(title_text='座位', tickvals=list(range(len(dev_names))), ticktext=dev_names)

# 设置布局
fig.update_layout(
    title='座位预约时间段分布',
    autosize=False,
    width=800,  # 调整图表宽度
    height=2000,  # 调整图表高度
    margin=dict(l=100, r=50, pad=4),
    yaxis=dict(scaleanchor="x", scaleratio=1)
)

# 显示图表
fig.show()

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# 创建一个文本输入框
input_box = widgets.Text(description='请输入内容：', continuous_update=False)

# 创建一个按钮组件
execute_button = widgets.Button(description='执行后续代码')

# 显示输入框和按钮
display(input_box)
display(execute_button)

# 定义按钮点击事件的处理函数
def execute_next_cell(button):
    global input_value
    input_value = input_box.value

# 将处理函数与按钮的点击事件关联
execute_button.on_click(execute_next_cell)

In [ ]:
print(f"输入框的值是：{input_value}")